In [5]:
import sys
sys.path.append('/Users/huibmeulenbelt/PycharmProjects/ufc/')
sys.path.append('/Users/huibmeulenbelt/PycharmProjects/ufc/scripts')
sys.path.append('/Users/huibmeulenbelt/PycharmProjects/ufc/scripts/markov')

In [6]:
import json
import numpy as np
from networkx.readwrite import json_graph
from typing import List

In [7]:
from fight import Fight

In [8]:
fight_id = 12120
PATH_G = f'/Users/huibmeulenbelt/PycharmProjects/ufc/scripts/markov/tests/{fight_id}.json'
G = json_graph.adjacency_graph(json.load(open(PATH_G)))

In [9]:
n_rounds = 3
fight = Fight(G=G, n_rounds=n_rounds)

In [27]:
k = 1           # tap out of blue
j = 33          # submission attempt / choke by fighter red

In [11]:
j = np.atleast_1d(j)

The goal is to compute the probability that the chain was in transient state $j$ at step $n - 1$ before it got absorbed in state $k$ at time $n$

$$
P(X_{n-1} = j, X_n = k)
$$

Let $\tau$ be the first hitting time of state $k$. Because $k$ is absorbing

$$
P(X_{n-1} = j, X_n = k) = P(X_{n-1} = j, X_n = k, \tau=n)
$$


By the definition of $\tau$, this is equivalent to

$$
P(X_0 \neq k, X_1 \neq k, ..., X_{n-1} = j, X_n = k)
$$

Let $Q^{(k)}$ be the restricted transition matrix where transitions between non-$k$ states are kept, and transitions to and from $k$ are removed and $\mu$ be the initial distribution restricted to non-$k$ states.

In [12]:
mask = np.ones(fight.n_states, dtype=bool)
mask[k] = False

# remove transitions from and to absorbing state k
Q  = fight.P[:, mask][:, :, mask]
mu = fight.mu[:, mask]

y = np.searchsorted(np.where(mask)[0], j)

Each multiplication by $Q^{(k)}$ simulates a one-step transition while avoing state $k$

In [13]:
t = Q.shape[1]

Q_powers = np.zeros((fight.n_samples, fight.n, t, t))
Q_powers[:, 0] = np.eye(t)[None, :, :]

for n in range(1, fight.n):
    Q_powers[:, n] = Q_powers[:, n - 1] @ Q

In [14]:
Q_y = np.take(Q_powers, y, axis=-1)

Combined, we can write:

$$
P(X_{n-1} = j, X_n = k, \tau = n) = \mu \cdot (Q^{(k)})^{n-1} \cdot P_{jk}
$$

In [15]:
p_Xt_equals_j = np.einsum("st, sntj -> sjn", mu, Q_y)
p_jk = fight.P[:, j, k]
p = p_Xt_equals_j * p_jk[:, :, None]

All of the above can be defined in one function

In [16]:
def p_X_tau_minus_1(mu: np.ndarray, P: np.ndarray, j: int | List[int], k: int, n_steps: int) -> np.ndarray:
    """
    Probabilities of being in transient state(s) j at step tau - 1 where tau <= n_steps is the time of absorption
    """
    j = np.atleast_1d(j)

    n_samples, n_states, _ = P.shape
    mask = np.ones(n_states, dtype=bool)
    mask[k] = False

    # remove transitions from and to absorbing state k
    Q  = P[:,  mask][:, :, mask]
    mu = mu[:, mask]

    y = np.nonzero(mask)[0].searchsorted(j)
    t = Q.shape[1]

    Q_powers = np.zeros((n_samples, n_steps, t, t))
    Q_powers[:, 0] = np.eye(t)[None, :, :]

    for n in range(1, n_steps):
        Q_powers[:, n] = Q_powers[:, n - 1] @ Q

    Q_y = np.take(Q_powers, y, axis=-1)
    p_Xt_equals_j = np.einsum("st, sntj -> sjn", mu, Q_y)
    p_jk = P[:, j, k]
    p = p_Xt_equals_j * p_jk[:, :, None]

    return np.sum(p, axis=2)

In [28]:
p_submission_by_red = p_X_tau_minus_1(
    mu=fight.mu,
    P=fight.P,
    j=j,
    k=k,
    n_steps=fight.n)

In [18]:
# check whether k is absorbing
absorbing = bool(np.all(fight.T[:, k, k] == 1))
print(f'k is absorbing: \n{absorbing}')

k is absorbing: 
True


In [19]:
# check whether k can only be reached through a single state
n_states_that_can_reach_k = np.sum(fight.T[:, :, k] != 0, axis=1)
single_entry = bool(np.all(n_states_that_can_reach_k - 1 == 1))                         # subtract 1 (self-loop of k)
print(f'k can only be reached through a single state: \n{single_entry}')

k can only be reached through a single state: 
True


In [29]:
# check whether k can only be reached through j
through_j = bool(np.all(fight.T[:, j, k] != 0))
print(f'k can only be reached through j: \n{through_j}')

k can only be reached through j: 
True


Logically, if is only possible to reach $k$ through $j$ then

$$
P(\text{absorbed in k after n steps}) = P(X_{n-1} = j, X_n = k, \tau = n)
$$

In [34]:
p_submission_red = fight.p_submission_red

In [39]:
equal = bool(np.allclose(p_submission_red, p_submission_by_red.squeeze(1)))
print(f'probabilities are equal: \n{equal}')

probabilities are equal: 
True


# Mutually-exclusive

For any fixed time $n$, the events:

$$
X_{n-1} = j, X_{\tau} = k, \tau = t \quad \text{for } j \in S'
$$

are mutually exclusive, because the chain can only be in one state at time $\tau - 1$, and it can only enter $k$ once.
Therefore, these events cannot occur simultaneously. Hence, the sum of their probabilities gives the total probability that the chain is absorbed in $k$ at time $n$:

$$
\sum_{j \in S'} P(X_{\tau-1} = j,\ X_{\tau} = k,\ \tau = t) = P(X_t = k, \tau = t)
$$

In [23]:
k = 0            # knockout of blue
j = [30, 32]     # ground strike landed by red, standing strike landed by red

In [24]:
p_knockout_by_red = p_X_tau_minus_1(
    mu=fight.mu,
    P=fight.P,
    j=j,
    k=k,
    n_steps=fight.n)

In [25]:
p_knockout_red = fight.p_knockout_red

In [26]:
equal = bool(np.allclose(p_knockout_red, np.sum(p_knockout_by_red, axis=1)))
print(f'probabilities are equal: \n{equal}')

probabilities are equal: 
True
